In [ ]:
import os
import json
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from vgg import vgg
from pathlib import Path

class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.classes = [d.name for d in os.scandir(img_dir) if d.is_dir()]  # 获取所有类别文件夹
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}  # 构建类别到索引的映射
        self.img_list = []  # 用于存储所有图像路径及其对应的类别索引

        for idx, cls in enumerate(self.classes):
            cls_dir = os.path.join(img_dir, cls)
            for img_name in os.listdir(cls_dir):
                if img_name.endswith('.jpg'):
                    img_path = os.path.join(cls_dir, img_name)
                    self.img_list.append((img_path, idx))  # 存储图像路径和其类别索引

        self.transform = transform
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path, label = self.img_list[idx]
        img = Image.open(img_path).convert("RGB") 
        if self.transform:
            img = self.transform(img)
        return img, label

def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print(f"Number of predictions: {len(all_preds)}, Number of labels: {len(all_labels)}")
    return all_preds, all_labels
    


def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # Load dataset
    test_dataset = CustomDataset(img_dir="//home//val", transform=data_transform)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Create and load model
    model_name = "vgg13"
    model = vgg(model_name=model_name, num_classes=2)
    model.to(device)
    weights_path ="//hy-tmp//vgg13Net.pth"
    model.load_state_dict(torch.load(weights_path, map_location=device))

    # Evaluate and print confusion matrix
    all_preds, all_labels = evaluate_model(model, test_loader, device)
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:\n", cm)
    print(f"Test dataset length: {len(test_dataset)}")


if __name__ == '__main__':
    main()
